In [ ]:
import os
import numpy as np
import librosa
import librosa.display
import soundfile as sf



In [ ]:
# Augmentation Functions
def subtle_time_stretch(audio, rate_range=(0.95, 1.05)):
    rate = np.random.uniform(*rate_range)
    return librosa.effects.time_stretch(audio, rate)

def subtle_pitch_shift(audio, sr, pitch_range=(-1, 1)):
    steps = np.random.uniform(*pitch_range)
    return librosa.effects.pitch_shift(audio, sr, n_steps=steps)

def add_low_amplitude_noise(audio, noise_factor=0.005):
    noise = np.random.randn(len(audio))
    return audio + noise_factor * noise

def dynamic_range_compression(audio, compression_factor=0.1):
    return audio * (1.0 + np.random.uniform(-compression_factor, compression_factor))

def frequency_masking(audio, sr, mask_ratio=0.1):
    # Convert to spectrogram
    S = librosa.stft(audio)
    S_db = librosa.amplitude_to_db(np.abs(S))
    num_bands = int(S_db.shape[0] * mask_ratio)
    band_start = np.random.randint(0, S_db.shape[0] - num_bands)
    S_db[band_start:band_start + num_bands, :] = -80  # Mask by setting low dB
    # Convert back to audio
    S = librosa.db_to_amplitude(S_db)
    return librosa.istft(S)



In [ ]:
# Augmentation Pipeline
def augment_heartbeat_audio(input_dir, output_dir, sr=22050, augmentations_per_file=3):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for class_name in os.listdir(input_dir):
        class_input_dir = os.path.join(input_dir, class_name)
        class_output_dir = os.path.join(output_dir, class_name)

        if not os.path.exists(class_output_dir):
            os.makedirs(class_output_dir)

        for file_name in os.listdir(class_input_dir):
            file_path = os.path.join(class_input_dir, file_name)
            audio, sr = librosa.load(file_path, sr=sr)

            # Save original audio (optional)
            sf.write(os.path.join(class_output_dir, file_name), audio, sr)

            for i in range(augmentations_per_file):
                augmented_audio = audio

                # Apply subtle augmentations randomly
                if np.random.rand() < 0.5:
                    augmented_audio = subtle_time_stretch(augmented_audio)
                if np.random.rand() < 0.5:
                    augmented_audio = subtle_pitch_shift(augmented_audio, sr)
                if np.random.rand() < 0.5:
                    augmented_audio = add_low_amplitude_noise(augmented_audio)
                if np.random.rand() < 0.5:
                    augmented_audio = dynamic_range_compression(augmented_audio)
                if np.random.rand() < 0.5:
                    augmented_audio = frequency_masking(augmented_audio, sr)

                # Save the augmented file
                augmented_file_name = f"{os.path.splitext(file_name)[0]}_aug{i}.wav"
                sf.write(os.path.join(class_output_dir, augmented_file_name), augmented_audio, sr)

                print(f"Augmented file {augmented_file_name} created for class {class_name}")
